## Ideal Vacation Spots

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as me
import numpy as np
import itertools
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Retrieve Weather Data From CSV

In [2]:
# reat weather data from created csv file from weather analysis
weather_csv = 'output_data/city_weather_data.csv'
weather_df = me.read_csv(weather_csv)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloud Cover,Wind Speed,Country
0,Kuala Terengganu,5.3302,103.1408,78.75,94,20,5.99,MY
1,Yellowknife,62.4560,-114.3525,57.72,77,20,3.44,CA
2,Buraidah,26.3260,43.9750,109.85,9,92,4.47,SA
3,Yangi Marg`ilon,40.4272,71.7189,99.32,23,1,4.61,UZ
4,Vaini,-21.2000,-175.2000,73.56,88,75,12.66,TO


### Humidity Heatmap

In [3]:
# configure gmapse with API key
gmaps.configure(api_key=g_key)

# convert humity to floating value and store
# drop any NaN responses if present
weather_df = weather_df.dropna()
humidity = weather_df['Humidity'].astype(float)

# store lat and long into variable
locs = weather_df[['Lat', 'Lng']].astype(float)

# create humidity heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locs, weights=humidity, dissipating=False, max_intensity=max(weather_df['Humidity']),
                             point_radius=2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### DataFrame Creation Based on Ideal Weather Conditions

In [14]:
# narrow dataframe to find ideal weather coniditions
ideal_weather_df = weather_df

# drop any rows that do not contain all three of the following coniditons (ideal weather)
# max temp no more than 80° but no less than 70°
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Max Temp'] <= 80) & (ideal_weather_df['Max Temp'] >= 70)]

# wind speed less than 10 mph
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Wind Speed'] < 10)]

# zero cloud cover
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Cloud Cover'] == 0)]

# drop rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# display dataframe
ideal_weather_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloud Cover,Wind Speed,Country
0,Brigantine,39.4101,-74.3646,78.03,75,0,4.61,US
1,Saint George,37.1041,-113.5841,75.63,77,0,4.61,US
2,Jamestown,42.0970,-79.2353,71.19,78,0,5.75,US
3,Arrondissement de Saint-Denis,48.9167,2.3333,75.24,52,0,9.22,FR
4,Makakilo City,21.3469,-158.0858,77.32,73,0,8.05,US
5,Castro,-24.7911,-50.0119,72.00,48,0,3.06,BR
6,Zanesville,39.9404,-82.0132,74.50,77,0,4.61,US
7,Nānākuli,21.3906,-158.1547,78.75,73,0,4.61,US
8,Fukuechō,32.6881,128.8419,78.22,76,0,6.33,JP
9,Hutchinson,38.0608,-97.9298,74.52,82,0,4.61,US


### Hotel Maps

In [6]:
# create hotels dataframe
hotels_df = ideal_weather_df

# main dictionary for parameters of search
params = {
    'radius': 5000,
    'types': 'lodging',
    'key': g_key
}

limit = 10

# iterate rows through each location in dataframe, max 10 locations
for index, row in itertools.islice(hotels_df.iterrows(), limit):
    # get lat and lng from dataframe
    lat = row['Lat']
    lng = row['Lng']

    # update location of each iteration
    params['location'] = f'{lat}, {lng}'

    # base url for google apis
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # make API request using call url
    response = requests.get(base_url, params=params).json()

    # retrieve results
    results = response['results']

    # attempt to get hotel name from index and add to 'Hotel Name' column
    try:
        print(f"The closest hotel to {row['City']} is {results[0]['name']}.")
        hotels_df.loc[index, 'Hotel Name'] = results[0]['name']

    # if unable to locate
    except (KeyError, IndexError):
        print(f'Unable to complete record. Checking next city.')

    # print separator
    print(f"-" * 35)

The closest hotel to Brigantine is Legacy Vacation Resorts - Brigantine Beach.
-----------------------------------
The closest hotel to Saint George is Best Western Plus Abbey Inn.
-----------------------------------
The closest hotel to Jamestown is Hampton Inn & Suites Jamestown.
-----------------------------------
The closest hotel to Arrondissement de Saint-Denis is ibis budget Paris Porte de Montmartre.
-----------------------------------
The closest hotel to Makakilo City is Marriott's Ko Olina Beach Club.
-----------------------------------
The closest hotel to Castro is CHACARA BAILLY.
-----------------------------------
The closest hotel to Zanesville is Holiday Inn Express & Suites Zanesville North, an IHG Hotel.
-----------------------------------
The closest hotel to Nānākuli is Camp Pālehua.
-----------------------------------
The closest hotel to Fukuechō is Goto Guesthouse Utojuku & SA Car Rental.
-----------------------------------
The closest hotel to Hutchinson is Ham

In [17]:
# drop excess columns and show dataframe results 
hotels_df.dropna(inplace=True)
hotels_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloud Cover,Wind Speed,Country,Hotel Name
0,Brigantine,39.4101,-74.3646,78.03,75,0,4.61,US,Legacy Vacation Resorts - Brigantine Beach
1,Saint George,37.1041,-113.5841,75.63,77,0,4.61,US,Best Western Plus Abbey Inn
2,Jamestown,42.0970,-79.2353,71.19,78,0,5.75,US,Hampton Inn & Suites Jamestown
3,Arrondissement de Saint-Denis,48.9167,2.3333,75.24,52,0,9.22,FR,ibis budget Paris Porte de Montmartre
4,Makakilo City,21.3469,-158.0858,77.32,73,0,8.05,US,Marriott's Ko Olina Beach Club
5,Castro,-24.7911,-50.0119,72.00,48,0,3.06,BR,CHACARA BAILLY
6,Zanesville,39.9404,-82.0132,74.50,77,0,4.61,US,"Holiday Inn Express & Suites Zanesville North,..."
7,Nānākuli,21.3906,-158.1547,78.75,73,0,4.61,US,Camp Pālehua
8,Fukuechō,32.6881,128.8419,78.22,76,0,6.33,JP,Goto Guesthouse Utojuku & SA Car Rental
9,Hutchinson,38.0608,-97.9298,74.52,82,0,4.61,US,Hampton Inn Hutchinson


In [18]:
# show hotel locations on previously generated heatmap

# template to add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# store the dataframe rows
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

# create hotel symbol layer
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# add marker layer to heatmap and display
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))